In [1]:
import os
import glob
import time
import shutil
import gc

import numpy as np
import matplotlib.pyplot as plt

import hapod as hp


In [ ]:
memory_avail_total = hp.get_memory_size()
memory_forbidden = 2**30
print(f"total available memory is {memory_avail_total / 2**30:.2f} GB")
print(f"but {memory_forbidden / 2**30:.2f} GB will be made unavailable")

serializer = hp.NumpySerializer()

dtype = np.float64
n_rows = 3600000
refresh_snapshots = True
refresh_chunks = True
refresh_hapod = refresh_snapshots or True
refresh_rand = refresh_snapshots or True

n_svd_max_cols = hp.get_max_svd_columns(n_rows, 
                                        memory_limit=memory_avail_total - memory_forbidden,
                                        dtype=dtype)
print(f"the largest matrix for SVD is {n_rows, n_svd_max_cols} using {dtype}")

n_chunk_max_cols = n_svd_max_cols // 2
print(f"the hapod can use chunks of {n_chunk_max_cols} columns")


In [ ]:
n_cols = n_chunk_max_cols * 16
snapshots_matrix_memory = hp.get_matrix_memory_footprint((n_rows, n_cols), dtype=dtype)
print(f"a snapshots matrix of size {n_rows, n_cols} would use {snapshots_matrix_memory / 2**30:.2f} GB of memory")
print(f"    SVD would use {hp.get_svd_memory_footprint((n_rows, n_cols), dtype=dtype) / 2**30:.2f} GB of memory")

n_chunks = hp.get_n_chunks_fulltree(n_cols, n_chunk_max_cols=n_chunk_max_cols)
print(f"for a balanced, full, merge tree, will need {n_chunks} chunks with maximum size {n_chunk_max_cols} >= {n_cols / n_chunks:.3f} average")


In [ ]:
n_randomized_svd_max_cols = hp.get_max_randomized_svd_samples((n_rows, n_cols),
                                  memory_limit=memory_avail_total - memory_forbidden,
                                  )
print(f"randomized SVD can use {n_randomized_svd_max_cols} samples")
print(f"which correspond to {hp.get_randomized_svd_memory_footprint((n_rows, n_cols), n_randomized_svd_max_cols, dtype=dtype) / 2**30:.2f} GB of memory")


In [18]:
from tests.test_base import get_test_matrix_full_rank

X_fullrank, U_fullrank, s_true = get_test_matrix_full_rank(n_rows= n_cols,
                                                            n_cols = n_cols,
                                                            dtype = dtype,
                                                            return_Us = True)


In [6]:
work_dir = "/scratch/lfabris/hapod_test"
os.makedirs(work_dir, exist_ok=True)


In [ ]:
snapshots_dir = os.path.join(work_dir, "snapshots")
print(f"simulating a snapshot matrix with size {(n_rows, n_cols)} under {snapshots_dir}")

if not os.path.isdir(snapshots_dir) or refresh_snapshots:
    print(f"create snapshots under {snapshots_dir}")
    shutil.rmtree(snapshots_dir)
    os.makedirs(snapshots_dir, exist_ok=True)

    print(
        f"storing {snapshots_matrix_memory / 2**30:.3f} GB worth of snapshots"
    )

    rng = np.random.default_rng()

    elapsed_snapshots = -time.perf_counter()
    snapshots_fnames = []
    for i in range(n_cols):
        x = np.concatenate([X_fullrank[:, i], 
                                np.zeros(n_rows-n_cols, dtype=dtype)], 
                            axis=0).reshape(-1, 1)
        snapshot_fname = os.path.join(snapshots_dir, f"snapshot_{i:04d}")
        snapshot_fname = serializer.store(x, snapshot_fname)

        snapshots_fnames.append(snapshot_fname)
    elapsed_snapshots += time.perf_counter()
    print(f"created {len(snapshots_fnames)} snapshot files in {elapsed_snapshots:.3f}")
else:
    snapshots_fnames = list(glob.glob(os.path.join(snapshots_dir, "*.npy")))
    print(f"found {len(snapshots_fnames)} snapshot files in {snapshots_dir}")
    

In [ ]:
chunks_dir = os.path.join(work_dir, "chunks")
print(f"simulating chunks of maximum size {n_rows, n_chunk_max_cols} under {chunks_dir}")

if not os.path.isdir(chunks_dir) or refresh_chunks:
    print(f"create chunks under {chunks_dir}")
    shutil.rmtree(chunks_dir)
    os.makedirs(chunks_dir, exist_ok=True)

    elapsed_chunks = -time.perf_counter()
    chunks_fnames = hp.make_chunks(
        snapshots_fnames,
        chunks_dir,
        n_chunks=n_chunks,
        serializer=serializer,
    )
    elapsed_chunks += time.perf_counter()
    print(f"created {len(chunks_fnames)} column chunks files in {elapsed_chunks:.3f}")
else:
    chunks_fnames = list(glob.glob(os.path.join(chunks_dir, "*.npy")))
    print(f"found {len(chunks_fnames)} column chunks files in {chunks_dir}")


In [ ]:
gc.collect()

In [10]:
if refresh_hapod:
    hapod_tmp_dir = os.path.join(work_dir, "tmp")

    elapsed_hapod = -time.perf_counter()
    U_hapod, s_hapod = hp.hapod(chunks_fnames,
                    chunk_rank_max=n_chunk_max_cols,
                    temp_work_dir=hapod_tmp_dir,
                    serializer=serializer,
                    verbose=True)
    elapsed_hapod += time.perf_counter()

    print(f"finished hapod in {elapsed_hapod:.3f}")
    print(f"    U.shape {U_hapod.shape}")
    print(f"    s_hapod.shape {s_hapod.shape}")

    np.save(os.path.join(work_dir, "U_hapod.npy"), U_hapod)
    np.save(os.path.join(work_dir, "s_hapod.npy"), s_hapod)

    U_hapod = None
    del U_hapod
else:
    s_hapod = np.load(os.path.join(work_dir, "s_hapod.npy"))


In [ ]:
gc.collect()

In [12]:
# n_randomized_svd_max_cols = 85

In [ ]:
if refresh_rand:
    elapsed_rand = -time.perf_counter()
    U_rand, s_rand = hp.randomized_pod(chunks_fnames,
                                    n_sources_samples=n_randomized_svd_max_cols,
                                    serializer=serializer,)
    elapsed_rand += time.perf_counter()

    print(f"finished rand in {elapsed_rand:.3f}")
    print(f"    U_rand.shape {U_rand.shape}")
    print(f"    s_rand.shape {s_rand.shape}")

    np.save(os.path.join(work_dir, "U_rand.npy"), U_rand)
    np.save(os.path.join(work_dir, "s_rand.npy"), s_rand)

    U_rand = None
    del U_rand
else:
    s_rand = np.load(os.path.join(work_dir, "s_rand.npy"))


In [14]:
fname = os.path.join(work_dir, "U_hapod.npy")
(_, n_cols_hapod), _ = serializer.peek(fname)
ortho_hapod = hp.singular_vectors_orthogonality(np.load(fname)[:n_cols], 
                                                U_fullrank[:, :n_cols_hapod])

fname = os.path.join(work_dir, "U_rand.npy")
(_, n_cols_rand), _ = serializer.peek(fname)
ortho_rand = hp.singular_vectors_orthogonality(np.load(fname)[:n_cols], 
                                                U_fullrank[:, :n_cols_rand])


In [ ]:
plt.semilogy(s_true, label="true")
plt.semilogy(s_hapod, label="hapod")
plt.semilogy(s_rand, label="rand")

plt.legend()

plt.xlim(-4 , 5 + max(n_randomized_svd_max_cols, n_chunk_max_cols))

plt.show()
plt.close()

plt.plot(ortho_hapod, label="hapod")
plt.plot(ortho_rand, label="rand")

plt.legend()

plt.show()
plt.close()
